In [4]:
from bs4 import BeautifulSoup
import getpass
from iso3166 import countries # ISO 3166-1 alpha-3 (3 letter country codes)
import numpy as np
import pandas as pd
import requests
from sqlalchemy import create_engine, text

# API
## World Health Organization

### Check API structure

In [32]:
# API structure
base_url = "https://ghoapi.azureedge.net/api/"
response = requests.get(base_url)
print(response.status_code)
print(response.text[:500])

200
{"@odata.context":"https://ghoapi.azureedge.net/api/$metadata","value":[{"name":"IndicatorDimension","kind":"EntitySet","url":"IndicatorDimension"},{"name":"DIMENSION","kind":"EntitySet","url":"DIMENSION"},{"name":"GOE_Q136","kind":"EntitySet","url":"GOE_Q136"},{"name":"GHE_YLDNUM","kind":"EntitySet","url":"GHE_YLDNUM"},{"name":"SA_0000001770_ARCHIVED","kind":"EntitySet","url":"SA_0000001770_ARCHIVED"},{"name":"FINPROTECTION_CATA_TOT_25_LEVEL_SH","kind":"EntitySet","url":"FINPROTECTION_CATA_TOT_


In [33]:
def list_indicators(base_url, terms_to_search):
    url = f"{base_url}/Indicator"
    r = requests.get(url, timeout=30)
    r.raise_for_status()
    indicators = r.json().get("value", [])
    hiv_inds = [i for i in indicators if any(k in i.get("IndicatorName","").upper() for k in terms_to_search)]
    return hiv_inds

In [34]:
# Indicators (endpoints in WHO's API) potentially related to HIV and PrEP
terms_to_search = [
        "HIV",
        "AIDS",
        "ART", 
        "ANTIRETROVIRAL", 
        "ANTIRETROVIRALS", 
        "PREP"
    ]
hiv_indicators = list_indicators(base_url, terms_to_search)

for i in hiv_indicators:
    print(i.get("IndicatorCode"), "-", i.get("IndicatorName"))

AMRGLASS_QA03 - EQA provided to local laboratories participating in GLASS
HIV_0000000015 - Number of pregnant women living with HIV who received antiretrovirals for preventing mother-to-child transmission
HIV_0000000016 - Number of pregnant women living with HIV who received antiretrovirals for preventing mother-to-child transmission
HIV_0000000022 - Estimated number of people eligible for antiretroviral therapy according to 2010 guidelines
Camp_gov_prog - Campaign was part of a comprehensive tobacco control programme
EQ_CONDOMUSE - People who identify the use of condoms as a way to avoid AIDS (%)
HIV_0000000006 - Number of people dying from HIV-related causes
HIV_0000000011 - Reported number of children receiving antiretroviral therapy
HIV_0000000012 - Reported number of children receiving antiretroviral therapy, month and year of report
HIV_0000000020 - Estimated number of pregnant women living with HIV
HIV_0000000021 - Estimated number of pregnant women living with HIV needing antir

In [35]:
# Indicators (endpoints in WHO's API) potentially related to other STIs
terms_to_search = [
        #"STI",
        "STI:",
        "STIS",
        "SEXUALLY TRANSMITTED",
        "GONORRHOEA",
        "CHLAMYDIA",
        "SYPHILIS"
    ]
sti_indicators = list_indicators(base_url, terms_to_search)

for i in sti_indicators:
    print(i.get("IndicatorCode"), "-", i.get("IndicatorName"))

CaseCS - Congenital syphilis number of cases, reported
PercposANC - Antenatal care attendees who were positive for syphilis (%), reported
PercposMSM - Men who have sex with men (MSM) with active syphilis (%)
PerctreatedANC - Antenatal care attendees positive for syphilis who received treatment (%), reported
PercposSW - Sex workers with active syphilis (%)
PerctestedANC - Women accessing antenatal care (ANC) services who were tested for syphilis (%), reported
RateCS - Congenital syphilis rate per 100 000 live births, reported
GASP_ISOLATESDSR_PROPORTION - Antimicrobial resistance or decreased susceptibiity (DS/R) to antimicrobials in Neisseria gonorrhoea isolates (WHO-GASP) (proportion)
STI_CHLAMYDIA_CASES_NUM - Reported cases of chlamydia (number)
TG_SYPHILISPREV_PERCENT - Transgender: Syphilis prevalence (percent)
STI_GONORRHOEA_INCIDENCE_PER1000 - Incident cases of gonorrhoea in individuals (per 1000)
SW_SYPHILISPREV_PERCENT - Sex workers: Syphilis prevalence (percent)
STI_CHLAMYDIA_

In [36]:
# Entry structure
def check_entry_structure(base_url, indicator):
    url = base_url + indicator
    response = requests.get(url).json()
    for key, value in response['value'][30].items():
        print(key)
        print(value)

In [37]:
check_entry_structure(base_url, "HIV_0000000012")

Id
5981646
IndicatorCode
HIV_0000000012
SpatialDimType
COUNTRY
SpatialDim
CUB
TimeDimType
YEAR
ParentLocationCode
AMR
ParentLocation
Americas
Dim1Type
None
TimeDim
2010
Dim1
None
Dim2Type
None
Dim2
None
Dim3Type
None
Dim3
None
DataSourceDimType
None
DataSourceDim
None
Value
Dec. 10
NumericValue
None
Low
None
High
None
Comments
None
Date
2013-06-11T14:01:25+02:00
TimeDimensionValue
2010
TimeDimensionBegin
2010-01-01T00:00:00+01:00
TimeDimensionEnd
2010-12-31T00:00:00+01:00


### Import data from API

In [38]:
def get_country_name(alpha3):
    try:
        return countries.get(alpha3).name
    except:
        return None

In [39]:
# For use when the indicator includes ungrouped data
def generate_indicator_df(indicator_reference):
    base_url = "https://ghoapi.azureedge.net/api/"
    url = base_url + indicator_reference
    response = requests.get(url).json()
    territories_types = []
    territories = []
    years = []
    numbers = []
    for territory in response['value']:
        territories_types.append(territory['SpatialDimType'])
        territories.append(territory['SpatialDim'])
        years.append(territory['TimeDim'])
        numbers.append(territory['NumericValue'])
    
    df = pd.DataFrame({
        "territory_type":territories_types,
        "territory":territories,
        "year":years,
        "value":numbers
    })

    df = df[df["territory_type"]=="COUNTRY"]
    df.drop(columns=["territory_type"], inplace=True)
    df.rename(columns={"territory":"country_code"}, inplace=True)
    df["country"] = df["country_code"].apply(get_country_name)
    df = df.iloc[:, [3, 0, 1, 2]]
    return df

In [40]:
# For use when the indicator includes data grouped by sex
def generate_indicator_df_bothsexes(indicator_reference):
    base_url = "https://ghoapi.azureedge.net/api/"
    url = base_url + indicator_reference
    response = requests.get(url).json()
    territories_types = []
    territories = []
    years = []
    numbers = []
    for territory in response['value']:
        if territory['Dim1'] == "SEX_BTSX":
            territories_types.append(territory['SpatialDimType'])
            territories.append(territory['SpatialDim'])
            years.append(territory['TimeDim'])
            numbers.append(territory['NumericValue'])
    
    df = pd.DataFrame({
        "territory_type":territories_types,
        "territory":territories,
        "year":years,
        "value":numbers
    })

    df = df[df["territory_type"]=="COUNTRY"]
    df.drop(columns=["territory_type"], inplace=True)
    df.rename(columns={"territory":"country_code"}, inplace=True)
    df["country"] = df["country_code"].apply(get_country_name)
    df = df.iloc[:, [3, 0, 1, 2]]
    return df

In [41]:
# For use when the indicator includes data grouped by sex and age
def generate_indicator_df_bothsexesages(indicator_reference):
    base_url = "https://ghoapi.azureedge.net/api/"
    url = base_url + indicator_reference
    response = requests.get(url).json()
    territories_types = []
    territories = []
    years = []
    numbers = []
    for territory in response['value']:
        if (territory['Dim1'] == "SEX_BTSX") & (territory['Dim2'] == "AGEGROUP_YEARSALL"):
            territories_types.append(territory['SpatialDimType'])
            territories.append(territory['SpatialDim'])
            years.append(territory['TimeDim'])
            numbers.append(territory['NumericValue'])
    
    df = pd.DataFrame({
        "territory_type":territories_types,
        "territory":territories,
        "year":years,
        "value":numbers
    })

    df = df[df["territory_type"]=="COUNTRY"]
    df.drop(columns=["territory_type"], inplace=True)
    df.rename(columns={"territory":"country_code"}, inplace=True)
    df["country"] = df["country_code"].apply(get_country_name)
    df = df.iloc[:, [3, 0, 1, 2]]
    return df

In [42]:
# HIV_0000000001 - Estimated number of people (all ages) living with HIV
estimated_total_HIVpos = generate_indicator_df("HIV_0000000001")
estimated_total_HIVpos.rename(columns={"value":"HIV_total"}, inplace=True)
estimated_total_HIVpos

,country,country_code,year,HIV_total
0,Palau,PLW,2019,NaN
2,New Zealand,NZL,2014,3000.0
3,Chile,CHL,2013,44000.0
4,Suriname,SUR,2020,7300.0
5,Jordan,JOR,2012,500.0
...,...,...,...,...
5090,Niue,NIU,2010,NaN
5091,Ukraine,UKR,2016,NaN
5092,India,IND,2013,NaN
5093,Honduras,HND,2016,20000.0


In [43]:
# HIV_0000000026 - Number of new HIV infections
new_infections = generate_indicator_df("HIV_0000000026")
new_infections.rename(columns={"value":"HIV_new"}, inplace=True)
new_infections

,country,country_code,year,HIV_new
0,Ireland,IRL,2015,500.0
1,Timor-Leste,TLS,2014,200.0
2,Suriname,SUR,2020,500.0
3,Israel,ISR,2019,NaN
4,Sao Tome and Principe,STP,2008,NaN
...,...,...,...,...
5090,Slovakia,SVK,2020,200.0
5091,Burkina Faso,BFA,2000,11000.0
5092,Jordan,JOR,2008,100.0
5093,Türkiye,TUR,2005,NaN


In [44]:
# SDGHIV - New HIV infections (per 1000 uninfected population)
new_infections_per1000 = generate_indicator_df_bothsexes("SDGHIV")
new_infections_per1000.rename(columns={"value":"HIV_new_per1000"}, inplace=True)
new_infections_per1000

,country,country_code,year,HIV_new_per1000
0,Panama,PAN,2004,0.37
1,North Macedonia,MKD,1990,0.01
2,El Salvador,SLV,1998,0.43
3,Uganda,UGA,1990,7.40
4,Guatemala,GTM,2009,0.10
...,...,...,...,...
7030,Bulgaria,BGR,2021,0.10
7031,Peru,PER,2015,0.18
7032,Poland,POL,2005,NaN
7033,Morocco,MAR,2013,0.10


In [45]:
# HIV_ARTCOVERAGE - Estimated antiretroviral therapy coverage among people living with HIV (%)
art_coverage = generate_indicator_df("HIV_ARTCOVERAGE")
art_coverage.rename(columns={"value":"ART_coverage_pct"}, inplace=True)
art_coverage

,country,country_code,year,ART_coverage_pct
0,Spain,ESP,2010,65.0
1,Guyana,GUY,2024,69.0
2,Latvia,LVA,2007,6.0
3,Fiji,FJI,2000,0.0
4,"Iran, Islamic Republic of",IRN,2005,NaN
...,...,...,...,...
5090,Viet Nam,VNM,2011,31.0
5091,Kuwait,KWT,2008,32.0
5092,Mauritius,MUS,2023,NaN
5093,Saudi Arabia,SAU,2006,30.0


In [46]:
# HIV_0000000027 - HIV tests performed where results were received by a person (testing volume), reported number
hiv_tests = generate_indicator_df_bothsexesages("HIV_0000000027")
hiv_tests.rename(columns={"value":"HIV_tests"}, inplace=True)
hiv_tests

,country,country_code,year,HIV_tests
0,"Venezuela, Bolivarian Republic of",VEN,2022,263500.0
1,Chad,TCD,2024,1031044.0
2,Guinea-Bissau,GNB,2024,NaN
3,Austria,AUT,2022,NaN
4,Ireland,IRL,2024,260355.0
...,...,...,...,...
577,Palau,PLW,2023,NaN
578,"Venezuela, Bolivarian Republic of",VEN,2023,378049.0
579,Liberia,LBR,2022,NaN
580,Gabon,GAB,2024,121269.0


In [47]:
# STI_CHLAMYDIA_CASES_NUM - Reported cases of chlamydia (number)
chlamydia = generate_indicator_df("STI_CHLAMYDIA_CASES_NUM")
chlamydia.rename(columns={"value":"chlamydia_cases"}, inplace=True)
chlamydia

,country,country_code,year,chlamydia_cases
0,Finland,FIN,2011,13666.0
1,France,FRA,2023,19122.0
2,Italy,ITA,2014,940.0
3,Iceland,ISL,2021,1807.0
4,United Kingdom of Great Britain and Northern I...,GBR,2015,229147.0
...,...,...,...,...
449,Luxembourg,LUX,2021,1136.0
450,Luxembourg,LUX,2010,2.0
451,Cyprus,CYP,2014,0.0
452,Ireland,IRL,2020,6901.0


In [48]:
# STI_SYPHILIS_CASES_NUM - Reported cases of syphilis (number)
syphilis = generate_indicator_df("STI_SYPHILIS_CASES_NUM")
syphilis.rename(columns={"value":"syphilis_cases"}, inplace=True)
syphilis.drop_duplicates(inplace=True)
syphilis

,country,country_code,year,syphilis_cases
0,Cyprus,CYP,2018,44.0
1,Luxembourg,LUX,2013,27.0
2,Croatia,HRV,2022,48.0
3,Netherlands,NLD,2017,1519.0
4,Estonia,EST,2011,66.0
...,...,...,...,...
873,Kazakhstan,KAZ,2019,3484.0
888,Belarus,BLR,2016,565.0
912,New Zealand,NZL,2022,508.0
915,New Zealand,NZL,2016,320.0


In [49]:
syphilis[syphilis['country']=='Spain'].sort_values(by=['year'])

,country,country_code,year,syphilis_cases
422,Spain,ESP,2010,3187.0
86,Spain,ESP,2011,3522.0
403,Spain,ESP,2012,3641.0
8,Spain,ESP,2013,3723.0
156,Spain,ESP,2014,3568.0
93,Spain,ESP,2015,3756.0
720,Spain,ESP,2016,3356.0
257,Spain,ESP,2017,4941.0
355,Spain,ESP,2018,4826.0
571,Spain,ESP,2019,4880.0


### Merge imported data

In [50]:
merged = estimated_total_HIVpos.merge(new_infections, on=["country_code", "country", "year"], how="outer")
merged = merged.merge(new_infections_per1000, on=["country_code", "country", "year"], how="outer")
merged = merged.merge(art_coverage, on=["country_code", "country", "year"], how="outer")
merged = merged.merge(hiv_tests, on=["country_code", "country", "year"], how="outer")
merged = merged.merge(chlamydia, on=["country_code", "country", "year"], how="outer")
merged = merged.merge(syphilis, on=["country_code", "country", "year"], how="outer")
merged

,country,country_code,year,HIV_total,HIV_new,HIV_new_per1000,ART_coverage_pct,HIV_tests,chlamydia_cases,syphilis_cases
0,Afghanistan,AFG,1990,NaN,NaN,0.01,NaN,NaN,NaN,NaN
1,Afghanistan,AFG,1991,NaN,NaN,0.01,NaN,NaN,NaN,NaN
2,Afghanistan,AFG,1992,NaN,NaN,0.01,NaN,NaN,NaN,NaN
3,Afghanistan,AFG,1993,NaN,NaN,0.01,NaN,NaN,NaN,NaN
4,Afghanistan,AFG,1994,NaN,NaN,0.01,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
6797,Zimbabwe,ZWE,2020,1300000.0,18000.0,1.20,88.0,NaN,NaN,NaN
6798,Zimbabwe,ZWE,2021,1300000.0,21000.0,1.40,89.0,NaN,NaN,NaN
6799,Zimbabwe,ZWE,2022,1300000.0,17000.0,1.10,93.0,1451171.0,NaN,NaN
6800,Zimbabwe,ZWE,2023,1300000.0,15000.0,0.95,94.0,1605740.0,NaN,NaN


In [51]:
merged[merged['country']=='Wales']

,country,country_code,year,HIV_total,HIV_new,HIV_new_per1000,ART_coverage_pct,HIV_tests,chlamydia_cases,syphilis_cases


# Import PrEP users CSV, and merge to previous table
## 2024 Q4 Global PrEP Tracker, by PrEP Watch, AVAC

In [5]:
def get_country_code(country_name):
    try:
        return countries.get(country_name).alpha3 
    except KeyError:
        return None

In [94]:
prep = pd.read_csv("data/2024_Q4-Global-PrEP-Tracker.csv", na_values=["-", " - ", "  -   "])

In [95]:
prep.columns

Index(['country', '2016_Q3', '2016_Q4', '2016 Net Cumulative', '2017_Q1',
       '2017_Q2', '2017_Q3', '2017_Q4', '2017 Net Cumulative', '2018_Q1',
       '2018_Q2', '2018_Q3', '2018_Q4', '2018 Net Cumulative', '2019_Q1',
       '2019_Q2', '2019_Q3', '2019_Q4', '2019 Net Cumulative ', '2020_Q1',
       '2020_Q2', '2020_Q3', '2020_Q4', '2020 Net Cumulative', '2021_Q1',
       '2021_Q2', '2021_Q3', '2021_Q4', '2021 Net Cumulative', '2022_Q1',
       '2022_Q2', '2022_Q3', '2022_Q4', '2022 Net Cumulative', '2023_Q1',
       '2023_Q2', '  2023_Q3  ', ' 2023_Q4 ', '2023 Net Cumulative', '2024_Q1',
       '2024_Q2', '2024_Q3', '2024_Q4', '2024 Net Cumulative'],
      dtype='object')

In [96]:
prep

,country,2016_Q3,2016_Q4,2016 Net Cumulative,2017_Q1,2017_Q2,2017_Q3,2017_Q4,2017 Net Cumulative,2018_Q1,...,2023_Q1,2023_Q2,2023_Q3,2023_Q4,2023 Net Cumulative,2024_Q1,2024_Q2,2024_Q3,2024_Q4,2024 Net Cumulative
0,Albania,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0,0,0,0,NaN
1,Antigua and Barbuda,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.0,5.0,5.0,5.0,NaN,5,5,5,5,NaN
2,Argentina,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,620.0,1921.0,1921.0,1921.0,1792.0,1921,3927,3927,3927,2006
3,Armenia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,43.0,43.0,43.0,43.0,43,77,77,77,34
4,Australia,NaN,318.0,318.0,318.0,3084.0,8225.0,13496.0,13178.0,14373.0,...,62632.0,62632.0,68251.0,68251.0,11257.0,68251,74597,80254,80254,12003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140,Uzbekistan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,253.0,253.0,253.0,253.0,253,448,448,448,195
141,Vietnam,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,777.0,...,56050.0,60821.0,66341.0,71204.0,20939.0,75748,80013,83825,83825,12621
142,Wales,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,200.0,...,3256.0,3256.0,3256.0,3256.0,NaN,3256,3256,3256,3256,NaN
143,Zambia,NaN,319.0,319.0,319.0,329.0,332.0,591.0,272.0,932.0,...,390454.0,467704.0,547230.0,619042.0,267835.0,697158,808847,892572,895685,276643


In [97]:
# Drop unnecessary columns
prep.drop(columns=["2016_Q3", 
                   "2017_Q1", "2017_Q2", "2017_Q3", 
                   "2018_Q1", "2018_Q2", "2018_Q3", 
                   "2019_Q1", "2019_Q2", "2019_Q3",
                   "2020_Q1", "2020_Q2", "2020_Q3",
                   "2021_Q1", "2021_Q2", "2021_Q3",
                   "2022_Q1", "2022_Q2", "2022_Q3",
                   "2023_Q1", "2023_Q2", "  2023_Q3  ",
                   "2024_Q1", "2024_Q2", "2024_Q3",
                   "2016 Net Cumulative",
                   "2017 Net Cumulative",
                   "2018 Net Cumulative",
                   "2019 Net Cumulative ",
                   "2020 Net Cumulative",
                   "2021 Net Cumulative",
                   "2022 Net Cumulative",
                   "2023 Net Cumulative",
                   "2024 Net Cumulative"
                  ], inplace=True)

# Rename remaining columns
prep.rename(columns={
    "2016_Q4":"2016_cumulative",
    "2017_Q4":"2017_cumulative",
    "2018_Q4":"2018_cumulative",
    "2019_Q4":"2019_cumulative",
    "2020_Q4":"2020_cumulative",
    "2021_Q4":"2021_cumulative",
    "2022_Q4":"2022_cumulative",
    " 2023_Q4 ":"2023_cumulative",
    "2024_Q4":"2024_cumulative",
}, inplace=True)

# Add country codes column
prep["country_code"] = prep["country"].apply(get_country_code)
prep = prep.iloc[:, [0, 10, 1, 2, 3, 4, 5, 6, 7, 8, 9]]

# Add missing country codes
prep.loc[prep["country"]=="Bosnia & Herzegovina", "country_code"] = "BIH"
prep.loc[prep["country"]=="Brunei", "country_code"] = "BRN"
prep.loc[prep["country"]=="Cote d'Ivoire", "country_code"] = "CIV"
prep.loc[prep["country"]=="DR Congo", "country_code"] = "COD"
prep.loc[prep["country"]=="Iran", "country_code"] = "IRN"
prep.loc[prep["country"]=="Kazakistan", "country_code"] = "KAZ"
prep.loc[prep["country"]=="Lao PDR", "country_code"] = "LAO"
prep.loc[prep["country"]=="Moldova", "country_code"] = "MDA"
prep.loc[prep["country"]=="South Korea", "country_code"] = "KOR"
prep.loc[prep["country"]=="Tanzania", "country_code"] = "TZA"
prep.loc[prep["country"]=="United States", "country_code"] = "USA"
prep.loc[prep["country"]=="Vietnam", "country_code"] = "VNM"

# Join UK in one row
uk_parts = prep[prep["country"].isin(["England", "Northern Ireland", "Scotland", "Wales"])]
uk_sum = uk_parts.drop(columns=["country", "country_code"]).sum(numeric_only=True)
uk_row = {"country": "United Kingdom", "country_code": "GBR"} 
uk_row.update(uk_sum.to_dict())
prep = pd.concat([prep, pd.DataFrame([uk_row])], ignore_index=True)
prep = prep[~prep["country"].isin(["England", "Northern Ireland", "Scotland", "Wales"])].reset_index(drop=True)

In [98]:
prep

,country,country_code,2016_cumulative,2017_cumulative,2018_cumulative,2019_cumulative,2020_cumulative,2021_cumulative,2022_cumulative,2023_cumulative,2024_cumulative
0,Albania,ALB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1,Antigua and Barbuda,ATG,NaN,NaN,NaN,NaN,NaN,5.0,5.0,5.0,5.0
2,Argentina,ARG,NaN,NaN,NaN,NaN,NaN,129.0,129.0,1921.0,3927.0
3,Armenia,ARM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,43.0,77.0
4,Australia,AUS,318.0,13496.0,17429.0,26520.0,37707.0,44798.0,56994.0,68251.0,80254.0
...,...,...,...,...,...,...,...,...,...,...,...
137,Uzbekistan,UZB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,253.0,448.0
138,Vietnam,VNM,NaN,NaN,2246.0,6735.0,14996.0,29027.0,50265.0,71204.0,83825.0
139,Zambia,ZMB,319.0,591.0,4155.0,27405.0,72309.0,202704.0,351207.0,619042.0,895685.0
140,Zimbabwe,ZWE,288.0,2965.0,9096.0,18398.0,27899.0,59918.0,143498.0,256459.0,337010.0


In [99]:
prep = prep.melt(
    id_vars=["country", "country_code"],   # keep these fixed
    var_name="year",                       # new column for year
    value_name="prep_users_cumulative"                # new column for values
)

# clean "year" column so it's just the number (strip "_cumulative")
prep["year"] = prep["year"].str.replace("_cumulative", "")
prep["year"] = prep["year"].astype(int)

In [100]:
prep

,country,country_code,year,prep_users_cumulative
0,Albania,ALB,2016,NaN
1,Antigua and Barbuda,ATG,2016,NaN
2,Argentina,ARG,2016,NaN
3,Armenia,ARM,2016,NaN
4,Australia,AUS,2016,318.0
...,...,...,...,...
1273,Uzbekistan,UZB,2024,448.0
1274,Vietnam,VNM,2024,83825.0
1275,Zambia,ZMB,2024,895685.0
1276,Zimbabwe,ZWE,2024,337010.0


# Web Scraping: 
## Year of approval of Truvada as PrEP per country - PrEP Country Data, PrEP Watch, AVAC

In [41]:
url = "https://www.prepwatch.org/data-by-country/"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
                  "(KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
    "Accept-Language": "en-US,en;q=0.9",
    "Referer": "https://www.google.com/",
    "Connection": "keep-alive",
    "Upgrade-Insecure-Requests": "1",
}
response = requests.get(url, headers=headers)
print(response)
soup = BeautifulSoup(response.content, "html.parser")

<Response [200]>


### Country list with links

In [42]:
country_links = {}
for a in soup.select("div.map__list.d-none ul li a"):
    country = a.get_text()
    country_link = a["href"]
    country_links[country]= country_link
country_links

{'Albania': 'https://www.prepwatch.org/countries/albania/',
 'Antigua & Barbuda': 'https://www.prepwatch.org/countries/antigua-barbuda/',
 'Argentina': 'https://www.prepwatch.org/countries/argentina/',
 'Armenia': 'https://www.prepwatch.org/countries/armenia/',
 'Australia': 'https://www.prepwatch.org/countries/australia/',
 'Austria': 'https://www.prepwatch.org/countries/austria/',
 'Azerbaijan': 'https://www.prepwatch.org/countries/azerbaijan/',
 'Bahamas': 'https://www.prepwatch.org/countries/bahamas/',
 'Bangladesh': 'https://www.prepwatch.org/countries/bangladesh/',
 'Barbados': 'https://www.prepwatch.org/countries/barbados/',
 'Belarus': 'https://www.prepwatch.org/countries/belarus/',
 'Belgium': 'https://www.prepwatch.org/countries/belgium/',
 'Belize': 'https://www.prepwatch.org/countries/belize/',
 'Benin': 'https://www.prepwatch.org/countries/benin/',
 'Bosnia & Herzegovina': 'https://www.prepwatch.org/countries/bosnia-herzegovina/',
 'Botswana': 'https://www.prepwatch.org/co

### Truvada as PrEP approval status and year for 1 country

In [43]:
url = "https://www.prepwatch.org/countries/spain/"
response = requests.get(url, headers=headers)
print(response)
soup = BeautifulSoup(response.content, "html.parser")

<Response [200]>


In [44]:
soup.find_all("div", string="Oral PrEP TDF/FTC — Gilead (Truvada)")[0].parent.select("span.approved-year")[0].get_text()

'2016'

In [45]:
soup.find_all("div", string="Oral PrEP TDF/FTC — Gilead (Truvada)")[0].parent.select("span.status__label.d-md-none")[0].get_text()

'Approved 2016'

### Iterate over list of countries

In [46]:
countries = []
truvada_prep_status = []
truvada_prep_year = []
for country, link in country_links.items():
    url = link
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")
    
    countries.append(country)
    try:
        truvada_prep_year.append(soup.find_all("div", string="Oral PrEP TDF/FTC — Gilead (Truvada)")[0].parent.select("span.approved-year")[0].get_text())
    except:
        truvada_prep_year.append(None)
    truvada_prep_status.append(soup.find_all("div", string="Oral PrEP TDF/FTC — Gilead (Truvada)")[0].parent.select("span.status__label.d-md-none")[0].get_text())
        
truvada_prep_adoption = pd.DataFrame({
    "country":countries,
    "status":truvada_prep_status,
    "prep_approval_year":truvada_prep_year
})

In [47]:
truvada_prep_adoption

,country,status,prep_approval_year
0,Albania,Approved,
1,Antigua & Barbuda,Data Not Available,
2,Argentina,Approved,
3,Armenia,Data Not Available,
4,Australia,Approved 2016,2016
...,...,...,...
140,Uzbekistan,Data Not Available,
141,Vietnam,Approved 2015,2015
142,Wales,Approved 2016,2016
143,Zambia,Approved 2017,2017


# Final tables merge, clean, and export

## Table including countries and years without null values in the columns HIV_total, HIV_new, prep_users_cumulative, chlamydia_cases, syphilis_cases, ART_coverage_pct.

In [101]:
final_table = merged.merge(prep, on=["country_code", "country", "year"], how="outer")
final_table = final_table[final_table["year"].isin(range(2010,2025))]

In [102]:
final_table

,country,country_code,year,HIV_total,HIV_new,HIV_new_per1000,ART_coverage_pct,HIV_tests,chlamydia_cases,syphilis_cases,prep_users_cumulative
20,Afghanistan,AFG,2010,4000.0,610.0,0.10,1.0,NaN,NaN,NaN,NaN
21,Afghanistan,AFG,2011,4500.0,670.0,0.10,3.0,NaN,NaN,NaN,NaN
22,Afghanistan,AFG,2012,4900.0,680.0,0.10,3.0,NaN,NaN,NaN,NaN
23,Afghanistan,AFG,2013,5400.0,790.0,0.10,4.0,NaN,NaN,NaN,NaN
24,Afghanistan,AFG,2014,5900.0,870.0,0.10,5.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
6937,Zimbabwe,ZWE,2020,1300000.0,18000.0,1.20,88.0,NaN,NaN,NaN,27899.0
6938,Zimbabwe,ZWE,2021,1300000.0,21000.0,1.40,89.0,NaN,NaN,NaN,59918.0
6939,Zimbabwe,ZWE,2022,1300000.0,17000.0,1.10,93.0,1451171.0,NaN,NaN,143498.0
6940,Zimbabwe,ZWE,2023,1300000.0,15000.0,0.95,94.0,1605740.0,NaN,NaN,256459.0


In [103]:
final_table = final_table.dropna(subset=['HIV_total', 
                                         'HIV_new', 
                                         'prep_users_cumulative', 
                                         'chlamydia_cases', 
                                         'syphilis_cases', 
                                         'ART_coverage_pct'])

final_table = final_table.drop(columns=["HIV_new_per1000", "HIV_tests"])

In [104]:
final_table['country'].unique()

array(['Australia', 'Belgium', 'Canada', 'Denmark', 'Spain', 'France',
       'Georgia', 'Greece', 'Croatia', 'Ireland', 'Iceland', 'Italy',
       'Lithuania', 'Luxembourg', 'Malta', 'Netherlands', 'New Zealand',
       'Slovenia'], dtype=object)

In [105]:
final_table['HIV_total'].sum()

3283120.0

In [106]:
final_table

,country,country_code,year,HIV_total,HIV_new,ART_coverage_pct,chlamydia_cases,syphilis_cases,prep_users_cumulative
306,Australia,AUS,2016,25000.0,880.0,81.0,94725.0,3380.0,318.0
307,Australia,AUS,2017,26000.0,840.0,83.0,101216.0,4402.0,13496.0
308,Australia,AUS,2018,27000.0,790.0,83.0,104763.0,5059.0,17429.0
309,Australia,AUS,2019,28000.0,730.0,84.0,107238.0,5900.0,26520.0
310,Australia,AUS,2020,29000.0,660.0,85.0,91327.0,5355.0,37707.0
...,...,...,...,...,...,...,...,...,...
5850,Slovenia,SVN,2019,800.0,100.0,81.0,397.0,54.0,70.0
5851,Slovenia,SVN,2020,840.0,100.0,84.0,280.0,31.0,70.0
5852,Slovenia,SVN,2021,880.0,100.0,82.0,369.0,37.0,70.0
5853,Slovenia,SVN,2022,910.0,100.0,83.0,412.0,34.0,70.0


In [107]:
final_table.to_csv("data/HIV_PrEP_data.csv")

## Table including all countries and years without null values in the columns HIV_new, prep_users_cumulative. Filtered by countries with better quality PrEP data.

In [108]:
filtered_table = merged.merge(prep, on=["country_code", "country", "year"], how="outer")
filtered_table = filtered_table[filtered_table["year"].isin(range(2010,2025))]

countries_to_filter = ['Australia',
    'Brazil',
    'Burundi',
    'Cameroon',
    "Cote d'Ivoire",
    'Eswatini',
    'Ethiopia',
    'Kenya',
    'Nigeria',
    'Rwanda',
    'South Africa',
    'Tanzania',
    'Thailand',
    'Uganda',
    'Vietnam',
    'Zambia',
    'Zimbabwe']

filtered_table = filtered_table.dropna(subset=['HIV_new'])

filtered_table = filtered_table.drop(columns=["HIV_new_per1000", "HIV_tests"])
filtered_table = filtered_table[filtered_table['country'].isin(countries_to_filter)]

In [109]:
filtered_table

,country,country_code,year,HIV_total,HIV_new,ART_coverage_pct,chlamydia_cases,syphilis_cases,prep_users_cumulative
300,Australia,AUS,2010,20000.0,1000.0,62.0,NaN,NaN,NaN
301,Australia,AUS,2011,21000.0,1000.0,65.0,NaN,NaN,NaN
302,Australia,AUS,2012,22000.0,1000.0,68.0,NaN,NaN,NaN
303,Australia,AUS,2013,23000.0,980.0,73.0,83819.0,1719.0,NaN
304,Australia,AUS,2014,24000.0,950.0,76.0,86799.0,2039.0,NaN
...,...,...,...,...,...,...,...,...,...
6937,Zimbabwe,ZWE,2020,1300000.0,18000.0,88.0,NaN,NaN,27899.0
6938,Zimbabwe,ZWE,2021,1300000.0,21000.0,89.0,NaN,NaN,59918.0
6939,Zimbabwe,ZWE,2022,1300000.0,17000.0,93.0,NaN,NaN,143498.0
6940,Zimbabwe,ZWE,2023,1300000.0,15000.0,94.0,NaN,NaN,256459.0


In [110]:
filtered_table.to_csv("data/filtered_countries.csv")